# Make Cutout

This notebook creates a cutout of a specific region (NGC 3379) from a larger FITS image and its inverse variance map. It also converts the inverse variance map to a sigma map.

In [1]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

ModuleNotFoundError: No module named 'astropy'

## Configuration

Define input files, target coordinates, and cutout parameters. **Note:** You must edit the file paths to match your local environment.

In [ ]:
# -------------------------------------------------
# Input brick files (you must edit these paths)
# -------------------------------------------------
image_file  = "legacysurvey-1619p125-image-r.fits.fz"
invvar_file = "legacysurvey-1619p125-invvar-r.fits.fz"

# -------------------------------------------------
# Target coordinates: NGC 3379 (M105)
# -------------------------------------------------
center = SkyCoord(ra=161.9567 * u.deg, dec=12.5817 * u.deg, frame="icrs")

# Cutout size on the sky
size = (200 * u.arcsec, 200 * u.arcsec)  # adjust if needed

# Output filenames
out_sci   = "NGC3379_r_cutout.fits"
out_sigma = "NGC3379_r_sigma_cutout.fits"

## Data Loading

Read the science image and inverse variance map from the FITS files. We also extract the WCS information.

In [ ]:
# -------------------------------------------------
# Read the brick images (note: data stored in HDU 1)
# -------------------------------------------------
with fits.open(image_file) as hdul_img, fits.open(invvar_file) as hdul_inv:
    img_hdu = hdul_img[1]      # science image
    inv_hdu = hdul_inv[1]      # inverse variance image

    data_img = img_hdu.data
    data_inv = inv_hdu.data

    # WCS from the science header
    wcs_full = WCS(img_hdu.header)

## Cutout Processing

1. Create 2D cutouts for both the science image and the inverse variance map.
2. Convert the inverse variance map to a sigma map using the relation $\sigma = 1 / \sqrt{\text{invvar}}$.

In [ ]:
    # -------------------------------------------------
    # 1. Make science and invvar cutouts
    # -------------------------------------------------
    cutout_img = Cutout2D(data_img, position=center, size=size, wcs=wcs_full)
    cutout_inv = Cutout2D(data_inv, position=center, size=size, wcs=wcs_full)

    # -------------------------------------------------
    # 2. Convert invvar → sigma map
    #    invvar = 1/sigma^2  => sigma = 1/sqrt(invvar)
    # -------------------------------------------------
    inv = cutout_inv.data
    sigma = np.full(inv.shape, np.nan, dtype=float)
    mask = inv > 0
    sigma[mask] = 1.0 / np.sqrt(inv[mask])

## Output

Write the resulting cutouts to new FITS files with the updated WCS header.

In [ ]:
    # -------------------------------------------------
    # 3. Write output FITS files with WCS
    # -------------------------------------------------
    hdr_cut = cutout_img.wcs.to_header()

    fits.PrimaryHDU(data=cutout_img.data, header=hdr_cut).writeto(out_sci, overwrite=True)
    fits.PrimaryHDU(data=sigma, header=hdr_cut).writeto(out_sigma, overwrite=True)

print("Wrote:", out_sci)
print("Wrote:", out_sigma)